### Great Hammerhead Shark Embryo Mercury Body Burden Calculation

Maternal offloading is the passive transfer of foreign sourced contaminants from mother to in-utero offspring, whether that be persistant organic pollutants, endocrine disruptors, heavy metals, or other toxicants. Historically notable environmental contaminants include DDT, microplastics, mercury, and lead.

Free, inorganic mercury $ (\text{Hg}_0) $ is a naturally-occurring, inert metal that can increase in bioavailability in ecological systems as a byproduct of human industrial activity (Beckers and Rinklebe 2017). When methylated, $ \text{Hg}_0 $ is transformed into its toxic organic form, methylmercury $ (\text{Hg - CH}_3) $, which is associated with detrimental health effects in both humans and wildlife (Krey et al. 2015). Measurements of Total Mercury (THg) include both $ \text{Hg}_0 $ and $ \text{Hg - CH}_3 $, with substantially larger contributions of $ \text{Hg - CH}_3 $

Maternal offloading of methylmercury likely poses detrimental developmental effects on the embryos of large sharks, including Great Hammerhead sharks (GHH). 

This notebook outlines a method of determining the total mass of mercury offloaded in GHH, as well as the utilizing of mathematical models for biological relevant smoothing of imprecise measurements.

The following morphometric measurements of GHH embryos are required to use this notebook:

- Litter Count: Litter_a, Litter_b, Litter_c... 

- Muscle Mercury Concentration (ppm)

- Liver Mercury Concentration (ppm)

- Total Mass (g)

- Liver Mass (g)



In [ ]:
import os
import pandas as pd
import numpy as np

os.chdir('/Users/RazWachtel/Desktop/GHH') # set your working directoy

# change when it works
Litters = pd.read_csv('Great Hammerhead Litters.csv') # load and read your local csv to the notebook

While no study has been conducted regarding the mass ratio between liver and muscle in GHH, a similar study has been conducted on the smallest member of the shared Sphyrnidae family, the Bonnethead shark (Lyons et al. 2021). 

The study resulted in the Bonnethead shark having a 70:30 mass ratio of muscle to liver in relation to total mass. Given similar developmental trajectory in-utero (Byrum et al. 2023), we can estimate the total body burden (mass) of mercury in unborn GHH embryos by applying this relationship to their respective muscle and liver concentrations of mercury:


$$ \text{Body Burden} = (L_{\text{mass}} \cdot L_{\text{THg}}) + (M_{\text{THg}} \cdot (0.7 \cdot T_{\text{mass}})) $$


where parameters included liver mass $ (L_{\text{mass}}) $, total liver mercury $ (L_{\text{THg}}) $, total mass $ (T_{\text{mass}}) $, and total muscle mercury $ (M_{\text{THg}}) $.  


In [ ]:
# Calculate Total Hg Body Burden 

Litter_a.loc[:, 'Hg Body Burden (µg)'] = Litter_a.apply(lambda row: ((pd.to_numeric(row['Mass'], errors='coerce') * 0.7) * 
                                            
                                            row['Muscle Mercury (ppm)']) +   # set (MTHg)(0.7 x Tmass)      

                                            (pd.to_numeric(row['Liver Mass'], errors='coerce') * 
                                             
                                             row['Liver Mercury (ppm)']) # add (Lmass x LTHg) 

                                    if pd.notna(row['Muscle Mercury (ppm)']) and pd.notna(row['Liver Mercury (ppm)']) and pd.notna(row['Liver Mass']) # only complete the function if individuals have all: MTHg, LThg, Lmass

                                    else pd.NA, 

                                    axis=1) # run through columns on df, not rows

print(Litter_a)

In [ ]:
Litter_a['Hg Body Burden (µg)'] = pd.to_numeric(Litter_a['Hg Body Burden (µg)'], errors='coerce') # ensure Total Hg is numeric 

Litter_a = Litter_a.dropna(subset=['Hg Body Burden (µg)']).sort_values(by='Hg Body Burden (µg)') # clean, sort dataset

print(Litter_a)

The second litter collected was massed using an imprecise scale, with 50 gram intervals. In order to more proxy the body burden in each individual, we must create biologically relevant mass scatter for these individuals.

We utilize a savitzky-golay filter with relevant parameters for this second litter below. 

In [ ]:
from scipy.signal import savgol_filter

signal_variance = Litter_b['Mass'].var() # find variance of the desired signal (mass)

# Smooth the "Mass" column using Savitzky-Golay filter
window_size, poly_order = 33, 3 # window size represents the number of points the filter will consider when fitting the data (use an odd number, less than or equal to # data points)
                                # poly order specifies the degree of polynomial to be fit to the window of points -- optimal as a smaller integer

# Apply filter to smooth signal
smoothed_signal = savgol_filter(Litter_a['Mass'], window_size, poly_order)

# Estimate the noise by subtracting the smoothed signal from the observed data
noise_estimate = Litter_b['Mass'] - smoothed_signal

# Calculate the variance of the noise estimate
noise_variance = noise_estimate.var()

# Calculate SNR
SNR = signal_variance / noise_variance

# Calculate SNR in dB
SNR_dB = 10 * np.log10(SNR)

# Find Standard Deviation of the variance
SDvar = np.sqrt(signal_variance / SNR)

print(f"Signal Variance: {signal_variance}")
print(f"Estimated Noise Variance: {noise_variance}")
print(f"SNR: {SNR}")
print(f"SNR (dB): {SNR_dB}")
print(f"SDvar: {SDvar}")

In [ ]:
set 
np.random.seed(1234) # set seed to maintain consistency in random numbers generated

noise = np.random.normal(0, SDvar, size=len(Litter_b)) # set mean to 0 (want to maintain the same mean of the original signal), with a SD of the calculated SDvar
                                                # size determines the amount of noise values, which we want equal to the amount of original values (unsmoothed)

Litter_b['Smoothed Mass'] = Litter_b['Mass'] + noise # add new column to dataframe: mass and calculated noise summed

Now that we've smoothed values for our dataset with imprecise mass measurements, we can now calculate a total mercury body burden within these subjects.

In [ ]:
# Calculate Total Hg Body Burden 

Litter_b.loc[:, 'Hg Body Burden (µg)'] = Litter_b.apply(lambda row: ((pd.to_numeric(row['Smoothed Mass'], errors='coerce') * 0.7) * 
                                            
                                            row['Muscle Mercury (ppm)']) +   # set (MTHg)(0.7 x Tmass)      

                                            (pd.to_numeric(row['Liver Mass'], errors='coerce') * 
                                             
                                             row['Liver Mercury (ppm)']) # add (Lmass x LTHg) 

                                    if pd.notna(row['Muscle Mercury (ppm)']) and pd.notna(row['Liver Mercury (ppm)']) and pd.notna(row['Liver Mass']) # only complete the function if individuals have all: MTHg, LThg, Lmass

                                    else pd.NA, 
                                    
                                    axis=1) # run through columns on df, not rows

Litter_b = Litter_b.dropna(subset = ['Hg Body Burden (µg)']).sort_values(by='Hg Body Burden (µg)')

print(Litter_b)

When a full litters' body burden is summed together, its result represents the amount of Hg physiologically removed from the litter's maternal source. 

In [ ]:
Litter_bodyburden = pd.concat([Litter_a, Litter_b], ignore_index='True') # merge data to one dataframe
print(Litter_bodyburden)
# Litter_bodyburden.to_csv('Litters With Mercury Body Burden', index=False) # export as csv to local device


Citations:

Beckers F, Rinklebe J. 2017. Cycling of mercury in the environment: Sources, fate, and human health implications: A review. Critical Reviews in Environmental Science and Technology. 47(9):693–794. doi:10.1080/10643389.2017.1326277.

Krey A, Ostertag SK, Chan HM. 2015. Assessment of neurotoxic effects of mercury in beluga whales (Delphinapterus leucas), ringed seals (Pusa hispida), and polar bears (Ursus maritimus) from the Canadian Arctic. Science of The Total Environment. 509–510:237–247. doi:10.1016/j.scitotenv.2014.05.134.

Lyons K, Adams DH, Bizzarro JJ. 2021. Evaluation of muscle tissue as a non-lethal proxy for liver and brain organic contaminant loads in an elasmobranch, the Bonnethead Shark (Sphyrna tiburo). Marine Pollution Bulletin. 167:112327. doi:10.1016/j.marpolbul.2021.112327.

Byrum SR, Frazier BS, Grubbs RD, Naylor GJP, Fraser GJ. 2023 Sep 28. Embryonic development in the bonnethead (Sphyrna tiburo) , a viviparous hammerhead shark. Developmental Dynamics.:dvdy.658. doi:10.1002/dvdy.658.

